In [ ]:
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 1: Install Libraries
##### Install boto3 to invoke Claude 3 Sonnet using Amazon’s boto3 library in Python.
##### Install pybase64 to covert images to base 64
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

In [3]:
pip install boto3, pybase, json

Note: you may need to restart the kernel to use updated packages.


In [ ]:
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 2: Import Libaries & Initiate Bedrock Runtime API
####### 2.1: Import boto3 to invoke Claude 3 Sonnet, 
####### 2.2: Import base64 to convert images,
####### 2.3: Import json to convert to JSON payload
####### 2.4: Import os for interacting with the operating system.
####### 2.5: Set home directory.
####### 2.6: Instantiate the Amazon Bedrock Runtime client.
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

In [11]:
# 2.1 - 2.4, Importing Libraries:
import boto3
import base64
import json
import os

# 2.5, Set home:
# If not in a project, set locally:
HOME = "/Users/nataliahelms/git/genai_research/"
# if in a project, use os (recommended):
#HOME = os.environ['HOME']

# 2.6, Instantiate the Bedrock Runtime Client with a region_name of us-east-1:
runtime = boto3.client("bedrock-runtime",region_name="us-east-1")

In [ ]:
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 3: Convert Images to base64
####### For Claude 3 Sonnet Model, the image is supplied using base64
####### So first, we need to convert our images to base64
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

In [ ]:
##### For Report App purposes, if we are able to access images of reports via URL, 
##### this functionality would be useful using the requests library:
#image_url = "https://url-to-the-image.com/file.jpg"
# Funtion to encode the image using URL
def encode_image_url(url):
    with requests.get(url).content as image_bytes
        return(base64.b64encode(image_bytes).decode("utf-8"))

In [12]:
##### However, for our immediate use case, we are using local files so we will use this method:
# Function to encode the image via path
def encode_image_path(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [13]:
# Getting the base64 string
orig_data_png = HOME + 'data/pdf_to_png/orig_data_1.png'
orig_data_base64_image = encode_image_path(orig_data_png)

In [ ]:
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 4: Build Inference Parameter Input
####### For Claude 3 Sonnet, we are required to use the Anthropic Claude Messages API as the inference parameter input.
####### Put this in the body parameter using json.dumps
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

In [15]:
# Step 4.1, set the body using paremeter input:
# json.dumps a dictionary of Claude’s required inference parameters
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": orig_data_base64_image,
                        },
                    },
                    {"type": "text", "text": "What is in this image?"},
                ],
            }
        ],
    }
)

In [ ]:
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 5: Invoke the Model
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

In [16]:
# Step 5, Invoke the model
response = runtime.invoke_model(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    body=body
)

NoCredentialsError: Unable to locate credentials

In [ ]:
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 6: Get the Response
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

In [ ]:
# Step 6,
# Convert the StreamingBody to a JSON encoded string using .read() 
# then turn the JSON string into a dictionary using json.loads
response_body = json.loads(response.get("body").read())

#The Claude JSON output includes a completion attribute with the text.
completion = response_body["completion"].strip()
print(completion)